In [1]:
import re, bisect
import matplotlib.pyplot as plt
import nltk
from collections import Counter
import seaborn as sns
import os
# Download Gutenberg corpus
nltk.download('gutenberg')
from nltk.corpus import gutenberg

# 文書読み込み
fileids = gutenberg.fileids()
docs = [gutenberg.raw(fid) for fid in fileids]

import numpy as np

[nltk_data] Downloading package gutenberg to
[nltk_data]     /Users/takatakiyugo/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [2]:
def tokenize_en(text):
    """英語テキストを小文字化して単語リストに分割する簡易トークナイザ"""
    return re.findall(r"[a-zA-Z]+", text.lower())


In [3]:
def show_postings(label, postings, k=60):
    print(f"{label}: df={len(postings)} head={postings[:k]}")


### 文書のチャンキング
500ワードごとに分割して文書数を増やす。RAGなどにも利用可能。

In [4]:
def chunk_documents(docs, chunk_size=500):
    chunks = []
    for doc in docs:
        words = tokenize_en(doc)
        for i in range(0, len(words), chunk_size):
            chunk = " ".join(words[i:i+chunk_size])
            chunks.append(chunk)
    return chunks

chunked_docs = chunk_documents(docs, 500)
print("Original docs:", len(docs))
print("Chunked docs:", len(chunked_docs))


Original docs: 18
Chunked docs: 4279


### Grep-like Search

### 文書のチャンキング
EDAは作品単位でしたが、検索以降は500ワードごとに分割した文書を使います。
これにより文書数が増え、RAG実習にも適した形になります。

In [5]:
def chunk_documents(docs, chunk_size=500):
    chunks = []
    for doc in docs:
        words = tokenize_en(doc)
        for i in range(0, len(words), chunk_size):
            chunk = " ".join(words[i:i+chunk_size])
            chunks.append(chunk)
    return chunks

chunked_docs = chunk_documents(docs, 500)
print("Original docs:", len(docs))
print("Chunked docs:", len(chunked_docs))

# チャンキング後の基礎統計
total_tokens = sum(len(tokenize_en(doc)) for doc in chunked_docs)
unique_tokens = len(set(w for doc in chunked_docs for w in tokenize_en(doc)))

print("Total tokens after chunking:", total_tokens)
print("Unique vocab after chunking:", unique_tokens)


Original docs: 18
Chunked docs: 4279
Total tokens after chunking: 2136080
Unique vocab after chunking: 41509


In [6]:
def grep_search_raw(term, docs):
    result = []
    for i, d in enumerate(docs):
        if term in d.lower():
            result.append(i)
    print(f"[Raw grep] Checked {len(docs)} docs in total")
    return result

def grep_search_token(term, docs):
    result = []
    for i, d in enumerate(docs):
        tokens = tokenize_en(d)
        if term in tokens:
            result.append(i)
    print(f"[Token grep] Checked {len(docs)} docs in total")
    return result

def grep_and_search(term1, term2, docs):
    res1 = set(grep_search_token(term1, docs))
    res2 = set(grep_search_token(term2, docs))
    return sorted(list(res1 & res2)) # 2つの結果を集合演算しているだけ



In [7]:
# 出力（df + 先頭30件）
res_raw_god = grep_search_raw("god", chunked_docs)
show_postings("Raw grep 'god'", res_raw_god)

res_tok_god = grep_search_token("god", chunked_docs)
show_postings("Token grep 'god'", res_tok_god)

res_tok_and = grep_and_search("love", "god", chunked_docs)
show_postings("Token grep AND 'love' AND 'god'", res_tok_and)


[Raw grep] Checked 4279 docs in total
Raw grep 'god': df=1752 head=[10, 11, 12, 13, 14, 15, 16, 30, 34, 35, 37, 39, 41, 43, 44, 70, 71, 72, 74, 82, 92, 93, 101, 115, 118, 120, 121, 137, 138, 139, 155, 191, 220, 221, 261, 263, 264, 270, 274, 282, 286, 301, 314, 316, 317, 325, 369, 396, 397, 400, 458, 480, 481, 488, 595, 600, 659, 662, 666, 692]
[Token grep] Checked 4279 docs in total
Token grep 'god': df=1609 head=[101, 261, 263, 264, 270, 274, 286, 314, 325, 369, 396, 397, 400, 458, 480, 481, 488, 595, 600, 659, 666, 692, 693, 697, 699, 703, 711, 735, 736, 737, 738, 739, 740, 741, 742, 743, 744, 745, 746, 751, 752, 753, 754, 758, 759, 760, 761, 762, 763, 764, 765, 766, 767, 770, 771, 772, 773, 774, 776, 777]
[Token grep] Checked 4279 docs in total
[Token grep] Checked 4279 docs in total
Token grep AND 'love' AND 'god': df=231 head=[264, 480, 481, 488, 666, 703, 770, 776, 841, 843, 910, 911, 1002, 1004, 1005, 1010, 1011, 1012, 1015, 1023, 1041, 1042, 1081, 1084, 1095, 1114, 1245, 1382, 

### raw grep

### token grep


df = document frequency : 何個の文書にこの単語が入っているか

dfがraw grepとtoken grepで違う

raw grepではgodivaなどでマッチする

token grepではそれがない

## 転置インデックス (array+bisect)

In [8]:
class InvertedIndexArray:
    def __init__(self):
        self.vocab = []       # ソート済み語彙リスト
        self.postings = {}    # term -> posting list

    def build(self, docs):
        vocab_set = set()
        postings = {}
        for doc_id, doc in enumerate(docs):
            for w in set(tokenize_en(doc)):
                vocab_set.add(w)
                postings.setdefault(w, []).append(doc_id)
        self.vocab = sorted(vocab_set)
        self.postings = {t: sorted(lst) for t, lst in postings.items()}

    # --- 自前の二分探索 ---
    def binary_search(self, arr, target):
        left, right = 0, len(arr) - 1
        while left <= right:
            mid = (left + right) // 2
            if arr[mid] == target:
                return mid
            elif arr[mid] < target:
                left = mid + 1
            else:
                right = mid - 1
        return -1  # not found

    def search(self, term):
        i = self.binary_search(self.vocab, term)
        if i != -1:
            return self.postings[self.vocab[i]]
        return []

    # --- AND検索 (1) set方式 ---
    def and_search_set(self, t1, t2):
        """集合演算ベース: 簡単・短いが、メモリ確保とO(n+m)コスト"""
        return sorted(set(self.search(t1)) & set(self.search(t2)))

    # --- AND検索 (2) マージ方式 ---
    def and_search_merge(self, t1, t2):
        """posting listがソート済みであることを利用した2ポインタ法"""
        p1 = self.search(t1)
        p2 = self.search(t2)
        i, j = 0, 0
        result = []
        while i < len(p1) and j < len(p2):
            if p1[i] == p2[j]:
                result.append(p1[i])
                i += 1
                j += 1
            elif p1[i] < p2[j]:
                i += 1
            else:
                j += 1
        return result

    def and_search_skip(p1, p2):
        n1, n2 = len(p1), len(p2)
        skip1 = int(math.sqrt(n1)) or 1
        skip2 = int(math.sqrt(n2)) or 1

        i, j = 0, 0
        result = []
        while i < n1 and j < n2:
            if p1[i] == p2[j]:
                result.append(p1[i]); i += 1; j += 1
            elif p1[i] < p2[j]:
                if (i + skip1 < n1) and (p1[i + skip1] <= p2[j]):
                    i += skip1
                else:
                    i += 1
            else:
                if (j + skip2 < n2) and (p2[j + skip2] <= p1[i]):
                    j += skip2
                else:
                    j += 1
        return result



In [9]:
inv_arr=InvertedIndexArray(); inv_arr.build(chunked_docs)


| 方法                                | メリット                                                                 | デメリット                                                                 |
| ----------------------------------- | ------------------------------------------------------------------------ | -------------------------------------------------------------------------- |
| **set方式** (`and_search_set`)        | - 実装が簡単で数行<br>- Python組込みの `set` に任せられる                                | - posting list を集合に変換するため順序情報を失う<br>- 内部で余計なメモリコピーとハッシュ計算が発生<br>- 大規模IRではほぼ使われない |
| **マージ方式** (`and_search_merge`)   | - posting list がソート済みであることを活用<br>- 計算量は O(n+m) で効率的<br>- 追加メモリはほぼ不要 | - 実装がやや長い<br>- posting list がソートされていないと使えない                                |
| **Skip List** (`and_search_skip`)     | - 長大な posting list の比較回数を減らせる<br>- 特に交差が少ない場合に高速<br>- Lucene など実システムでも採用 | - 実装が複雑<br>- skip pointer の設計（間隔など）に工夫が必要                                   |


skip listについては第1回授業資料を確認

googleなどはFSTを使っていると言われている

In [10]:
show_postings("Array+bisect 'love'", inv_arr.search("love"))
show_postings("Array+bisect 'god'", inv_arr.search("god"))
show_postings("Array+bisect AND 'love' & 'god'", inv_arr.and_search_merge("love","god"))


Array+bisect 'love': df=649 head=[6, 7, 18, 23, 24, 25, 28, 30, 36, 37, 38, 40, 42, 46, 48, 49, 50, 54, 55, 56, 57, 58, 60, 61, 62, 67, 72, 73, 74, 81, 83, 85, 88, 89, 90, 93, 98, 109, 117, 120, 132, 133, 134, 135, 136, 138, 140, 142, 144, 148, 149, 170, 172, 173, 174, 178, 191, 207, 208, 215]
Array+bisect 'god': df=1609 head=[101, 261, 263, 264, 270, 274, 286, 314, 325, 369, 396, 397, 400, 458, 480, 481, 488, 595, 600, 659, 666, 692, 693, 697, 699, 703, 711, 735, 736, 737, 738, 739, 740, 741, 742, 743, 744, 745, 746, 751, 752, 753, 754, 758, 759, 760, 761, 762, 763, 764, 765, 766, 767, 770, 771, 772, 773, 774, 776, 777]
Array+bisect AND 'love' & 'god': df=231 head=[264, 480, 481, 488, 666, 703, 770, 776, 841, 843, 910, 911, 1002, 1004, 1005, 1010, 1011, 1012, 1015, 1023, 1041, 1042, 1081, 1084, 1095, 1114, 1245, 1382, 1427, 1496, 1497, 1502, 1511, 1517, 1533, 1534, 1549, 1560, 1563, 1568, 1570, 1579, 1613, 1619, 1622, 1690, 1695, 1700, 1702, 1704, 1709, 1747, 1813, 1827, 1845, 1873, 1

token grepのdfと同じことから確認できる。

### Patricia Tree Implementation (Explained)
Patricia Tree is a compressed trie structure. We merge common prefixes to reduce node count.

In [11]:
class PatriciaNode:
    def __init__(self):
        self.children = {}
        self.is_word = False
        self.postings = []

class PatriciaTree:
    def __init__(self):
        self.root = PatriciaNode()

    def insert(self, word, doc_id):
        node = self.root
        w = word
        while w:
            for k in list(node.children.keys()):
                prefix = os.path.commonprefix([k, w])
                if prefix:
                     if prefix != k:
                        child = node.children.pop(k)
                        new_node = PatriciaNode()
                        new_node.children[k[len(prefix):]] = child
                        node.children[prefix] = new_node
                        node = new_node
                     else:
                        node = node.children[k]
                     w = w[len(prefix):]
                     break
            else:
                node.children[w] = PatriciaNode()
                node = node.children[w]
                w = ""
        node.is_word = True
        if doc_id not in node.postings:
            node.postings.append(doc_id)

    def search(self, word):
        node = self.root
        w = word
        while w:
            for k, child in node.children.items():
                if w.startswith(k):
                    w = w[len(k):]
                    node = child
                    break
            else:
                return []
        return sorted(node.postings) if node.is_word else []

    def starts_with(self, prefix):
        node = self.root
        w = prefix
        while w:
            for k, child in node.children.items():
                if w.startswith(k):
                    w = w[len(k):]
                    node = child
                    break
                elif k.startswith(w):
                    return True
            else:
                return False
        return True

# 全 chunked_docs から構築
tree = PatriciaTree()
for doc_id, doc in enumerate(chunked_docs):
    for w in set(tokenize_en(doc)):
        tree.insert(w, doc_id)


トライ木を作るが、例えば、god - iva
のようにノードを圧縮する

In [12]:

# 出力（df + head30）
show_postings("Patricia search 'love'", tree.search("love"))
show_postings("Patricia search 'god'", tree.search("god"))
show_postings("Patricia AND 'love' & 'god'", sorted(set(tree.search("love")) & set(tree.search("god"))))

Patricia search 'love': df=649 head=[6, 7, 18, 23, 24, 25, 28, 30, 36, 37, 38, 40, 42, 46, 48, 49, 50, 54, 55, 56, 57, 58, 60, 61, 62, 67, 72, 73, 74, 81, 83, 85, 88, 89, 90, 93, 98, 109, 117, 120, 132, 133, 134, 135, 136, 138, 140, 142, 144, 148, 149, 170, 172, 173, 174, 178, 191, 207, 208, 215]
Patricia search 'god': df=1609 head=[101, 261, 263, 264, 270, 274, 286, 314, 325, 369, 396, 397, 400, 458, 480, 481, 488, 595, 600, 659, 666, 692, 693, 697, 699, 703, 711, 735, 736, 737, 738, 739, 740, 741, 742, 743, 744, 745, 746, 751, 752, 753, 754, 758, 759, 760, 761, 762, 763, 764, 765, 766, 767, 770, 771, 772, 773, 774, 776, 777]
Patricia AND 'love' & 'god': df=231 head=[264, 480, 481, 488, 666, 703, 770, 776, 841, 843, 910, 911, 1002, 1004, 1005, 1010, 1011, 1012, 1015, 1023, 1041, 1042, 1081, 1084, 1095, 1114, 1245, 1382, 1427, 1496, 1497, 1502, 1511, 1517, 1533, 1534, 1549, 1560, 1563, 1568, 1570, 1579, 1613, 1619, 1622, 1690, 1695, 1700, 1702, 1704, 1709, 1747, 1813, 1827, 1845, 1873,


### 結果の一貫性チェック（chunk 単位）  
`grep（単語ベース）` / `Patricia`  の **posting list（チャンクIDの集合）が一致** していることを確認します。  
長大な配列は見づらいので、件数（df）と先頭数件だけを表示します。


In [13]:
def summarize(name, lst, k=30):
    print(f"{name}: df={len(lst)} head={lst[:k]}")

def compare_across(term):
    g = grep_search_token(term, chunked_docs)
    p = tree.search(term)
    print(f"=== term: '{term}' ===")
    summarize("grep(token)", g)
    summarize("patricia   ", p)
    print("Sets equal:", set(g) == set(p) )
    print()

def compare_and(term1, term2):
    g = sorted(set(grep_search_token(term1, chunked_docs)) & set(grep_search_token(term2, chunked_docs)))
    p = sorted(set(tree.search(term1)) & set(tree.search(term2)))
    print(f"=== AND: '{term1}' AND '{term2}' ===")
    summarize("grep(token)", g)
    summarize("patricia   ", p)
    print("Sets equal:", set(g) == set(p) )
    print()

compare_across("love")
compare_across("god")
compare_and("love", "god")


[Token grep] Checked 4279 docs in total
=== term: 'love' ===
grep(token): df=649 head=[6, 7, 18, 23, 24, 25, 28, 30, 36, 37, 38, 40, 42, 46, 48, 49, 50, 54, 55, 56, 57, 58, 60, 61, 62, 67, 72, 73, 74, 81]
patricia   : df=649 head=[6, 7, 18, 23, 24, 25, 28, 30, 36, 37, 38, 40, 42, 46, 48, 49, 50, 54, 55, 56, 57, 58, 60, 61, 62, 67, 72, 73, 74, 81]
Sets equal: True

[Token grep] Checked 4279 docs in total
=== term: 'god' ===
grep(token): df=1609 head=[101, 261, 263, 264, 270, 274, 286, 314, 325, 369, 396, 397, 400, 458, 480, 481, 488, 595, 600, 659, 666, 692, 693, 697, 699, 703, 711, 735, 736, 737]
patricia   : df=1609 head=[101, 261, 263, 264, 270, 274, 286, 314, 325, 369, 396, 397, 400, 458, 480, 481, 488, 595, 600, 659, 666, 692, 693, 697, 699, 703, 711, 735, 736, 737]
Sets equal: True

[Token grep] Checked 4279 docs in total
[Token grep] Checked 4279 docs in total
=== AND: 'love' AND 'god' ===
grep(token): df=231 head=[264, 480, 481, 488, 666, 703, 770, 776, 841, 843, 910, 911, 1002,


## 検索手法のまとめ（英語コーパス・チャンク単位）

- **Raw grep（部分一致）**  
  文字列の部分一致でヒット判定。単語境界を無視するためノイズが混ざりやすい。計算量は文書数に対して **O(N)**。

- **Token grep（単語一致）**  
  簡易トークナイザで分割し、単語集合に対して線形探索。結果は正確だが依然 **O(N)**。

- **転置インデックス（array + binary search）**  
  `term → posting list(docIDの昇順配列)` を構築。語彙はソートして二分探索（**O(log |V|)**）、
  posting list は二分探索や交差（AND）で利用（**O(log |P|)** または **O(|P1|+|P2|)**）。
  検索コストが小さく、実用的。

- **Patricia木（圧縮Trie, 接頭辞検索のデモ）**  
  文字列を辺ラベル単位で前方圧縮。**接頭辞検索**が高速（補完・辞書用途に有効）。
  本ノートでは構造理解を主目的とし、posting の正確性は **転置インデックス／** を基準にする。


> 以降の実装演習（圧縮・TF-IDF・RAGなど）は、転置インデックスを基盤として進めます。
